# Bidder class

Maybe leave the updating of the balances up to the Auction class

In [4]:
import numpy as np

In [5]:
class Bidder():
    """This class creates bidders equipped with knowledge of the number of users in the auction and the number 
    of auction rounds to be played"""
    
    def __init__(self, num_users, num_rounds):
        # each user has a list associated with their user_id: first entry is num_clicks, second is num_ads_shown,
        # third is the quotient of the two representing the best estimate of their clicking likelihood
        self.prob_estimates = {i:[0,0,0] for i in range(num_users)}
        self.prices_by_user = {i:[] for i in range(num_users)}
        self.current_user = 0
    
    def bid(self, user_id):
        """Returns a non-negative amount of money. If you don't wish to bid anything on a given user, this should 
        return 0.
        This is where at least some of the main logic will need to go. Maybe not the core explore/exploit strategy, 
        but at least the consequence of that strategy."""
        # going to start by creating a stupid bidder that has no concept of explore/exploit strategy
        # this bidder will not consider the relative probabilities of different users choosing to click
        self.current_user = user_id
        
        return np.random.uniform()

    
    def notify(self, auction_winner, price, clicked=None):
        """Used to send info about what happened in a round back to the bidder. auction_winner will be a boolean:
        True if the bidder won a given round, False otherwise. price will be the amount of the second bid, only the
        winner pays this. clicked will only take on a value if auction_winner is True (if the bidder won that given
        round) in which case it will take on a boolean value: True if the user clicked on the ad and False 
        otherwise"""
        if auction_winner:        # If the bidder won the auction
            if clicked:           # Additionally, if the user clicked
                # update the prob_estimates dictionary for the current user
                self.prob_estimates[self.current_user][0] += 1
                self.prob_estimates[self.current_user][1] += 1
                self.prob_estimates[self.current_user][2] = self.prob_estimates[self.current_user][0] / self.prob_estimates[self.current_user][1]
            else:  # if the user didn't click
                # update the prob_estimates dictionary for the current user
                self.prob_estimates[self.current_user][1] += 1
                self.prob_estimates[self.current_user][2] = self.prob_estimates[self.current_user][0] / self.prob_estimates[self.current_user][1]
        # record the winning price and attach it to the appropriate user for each round, what do OTHER people think that user is worth?
        self.prices_by_user[self.current_user].append(price)
                
    
    

In [19]:
# create a bidder in an auction with 5 users and 3 rounds

bidder1 = Bidder(5, 3) 

Let's take a look at the instance attributes as they are upon creation

In [20]:
# {balance} and {prob_estimates} attributes

print("Balance of bidder1:", bidder1.balance)
print("bidder1's understanding of the clicking likelihoods of the users: \n", bidder1.prob_estimates)
print("Default beginner user:", bidder1.current_user)

Balance of bidder1: 0
bidder1's understanding of the clicking likelihoods of the users: 
 {0: [0, 0, 0], 1: [0, 0, 0], 2: [0, 0, 0], 3: [0, 0, 0], 4: [0, 0, 0]}
Default beginner user: 0


In [21]:
# let's call .bid(), this should only change the current_user attribute and also it should return 0.5

bidder1.bid(3)

0.6580197531535115

In [22]:
# confirm that current_user attribute is now the same as the argument passed to the .bid() method

bidder1.current_user

3

In [23]:
# now call the .notify() method on bidder1 to let them know whether their bid and possibly subsequent ad was/were 
# successful

bidder1.notify(True, 0.60, True)

In [24]:
# now check to confirm that the relevant attributes have been updated

print("Balance of bidder1:", bidder1.balance)
print("bidder1's understanding of the clicking likelihoods of the users: \n", bidder1.prob_estimates)
print("Default beginner user:", bidder1.current_user)

Balance of bidder1: 0.4
bidder1's understanding of the clicking likelihoods of the users: 
 {0: [0, 0, 0], 1: [0, 0, 0], 2: [0, 0, 0], 3: [1, 1, 1.0], 4: [0, 0, 0]}
Default beginner user: 3


Awesome!! At this point we have a very simple bidder. It is simple because it only bids 50 cents every time. However, it does have a way to keep track of how much money it has and the relative clicking likelihoods of all the users in the game. Those will be critical for building an intelligent bidding strategy.